In [1]:
import os
from huggingface_hub import hf_hub_download

In [2]:
import numpy as np

In [3]:
local_model_directory="D:\Python\AI_POC\AI_POC2" # mention the localtion where you want to save the model



In [4]:
from transformers import pipeline





In [5]:
pipe = pipeline("text-generation", model=local_model_directory)

In [53]:
table_schema = """CREATE TABLE ENEX_MASTER  --table for entry exit 
(
  EMA_SEQ                  NUMBER(5),
  SPM_PERSON_NO            NUMBER(12),
  NAT_CODE                 NUMBER(3),
  EBO_CODE                 NUMBER(4),
  ETT_CODE                 NUMBER(3),
  SFU_CODE                 NUMBER(2)            DEFAULT 5,
  SDE_CODE                 NUMBER(3),
  EMA_TYPE                 NUMBER(1),
  EMA_YEAR                 NUMBER(4),
  EMA_DATE                 DATE,
  EMA_CREATED_BY           VARCHAR2(10 BYTE),
  EMA_DEL_FLAG             NUMBER(1)            DEFAULT 0,
  EMA_FLIGHT_TRIP_NO       VARCHAR2(10 BYTE),
  EMA_AIR_SHIP_CODE        NUMBER(10),
  AIRPORT_CODE             VARCHAR2(3 BYTE)
  );

CREATE TABLE NATIONALITIES
(
  CODE                NUMBER(3)        ,
  COUNTRY             VARCHAR2(50 BYTE),
  COUNTRY_ENG         VARCHAR2(50 BYTE),
  SHORT_DESC          VARCHAR2(3 BYTE)
  );

CREATE TABLE PERSON_MASTER   --table for person details
(
  SPM_PERSON_NO            NUMBER(12)          ,--foreign key to column SPM_PERSON_NO  in enex_master
  SU_ID                    VARCHAR2(10 BYTE)   ,
  NAT_CODE_CURR_NAT        NUMBER(3)           , --This is Nationality of person
  SFI_CODE                 NUMBER(3),
  EL_CODE                  NUMBER(3),
  EMI_CODE                 NUMBER(3),
  PRO_CODE                 NUMBER(7),
  NAT_CODE_RESI_IN         NUMBER(3),
  TRI1_CODE                NUMBER(5),
  NAT_CODE_PRE_NAT         NUMBER(3),
  NAT_CODE_BORN_IN         NUMBER(3),
  CIT_CODE                 NUMBER(3),
  SPM_FULL_ANAME           VARCHAR2(80 BYTE),
  SPM_ANAME                VARCHAR2(25 BYTE),
  SPM_FATHER_ANAME         VARCHAR2(25 BYTE),
  SPM_GRAND_ANAME          VARCHAR2(25 BYTE),
  SPM_POST_GRAND_ANAME     VARCHAR2(25 BYTE),
  SPM_FIFTH_ANAME          VARCHAR2(25 BYTE),
  SPM_FAMILY_ANAME         VARCHAR2(25 BYTE),
  SPM_MOTHER_FATHER_ANAME  VARCHAR2(15 BYTE),
  SPM_FULL_ENAME           VARCHAR2(80 BYTE),
  SPM_ENAME                VARCHAR2(25 BYTE),
  SPM_FATHER_ENAME         VARCHAR2(25 BYTE),
  SPM_GRAND_ENAME          VARCHAR2(25 BYTE),
  SPM_POST_GRAND_ENAME     VARCHAR2(25 BYTE),
  SPM_FIFTH_ENAME          VARCHAR2(25 BYTE),
  SPM_FAMILY_ENAME         VARCHAR2(25 BYTE),
  SPM_CLAN_ENAME           VARCHAR2(20 BYTE),
  SPM_MOTHER_ENAME         VARCHAR2(15 BYTE),
  SPM_MOTHER_FATHER_ENAME  VARCHAR2(15 BYTE),
  SPM_DOB                  DATE,
  SPM_PERSON_CLASS         NUMBER(3),
  SPM_GENDER               NUMBER(1),
  SPM_MARITAL_STATUS       NUMBER(1),
  SPM_PLACE_OF_BIRTH       VARCHAR2(80 BYTE),
  SPM_NATIONAL_ID          VARCHAR2(20 BYTE),
  SPM_PLACE_OF_BIRTH_ENG   VARCHAR2(80 BYTE),
  CID_CLASS                NUMBER(3)
;
"""

question = "Total Number of persons ?"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

In [54]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(local_model_directory)
model = AutoModelForCausalLM.from_pretrained(local_model_directory)

In [55]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=1600)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
output = 'SELECT' + output.split('SELECT')[-1]

print(output)



SELECT COUNT(


In [56]:
def new_question(p_question):
 

  prompt = f"""{table_schema}
  -- Using valid SQLite, answer the following questions for the tables provided above.
  -- {p_question}
  SELECT"""
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  generated_ids = model.generate(input_ids, max_length=4600)
  output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  output = 'SELECT' + output.split('SELECT')[-1]
  return output
  print(output)

In [36]:
new_question('Total  count from entry exit details')

'SELECT COUNT(*) FROM ENEX_MASTER;'

In [37]:
new_question('entry exit details of person no 1237614')

'SELECT * FROM ENEX_MASTER WHERE ESM_PERSON_NO = 12376'

In [57]:
new_question('Nationality of passenger with person no 1237614')

'SELECT T1.SPM_FAMILY_ENAME FROM PERSON_MASTER AS T1 JOIN SSPERSON_MASTER AS T2 ON T1.SPM_PERSON_NO = T2.SPM_PERSON_NO WHERE T2.SPM_PERSON_NO = 1237614;'

In [51]:
new_question('passenger name of person number 1237614')

'SELECT ESM_FULL_ENAME FROM PERSON_MASTER WHERE SPM_PERSON_NO = 1237614;'